In [25]:
!pip install transformers==4.37.1


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [26]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline,AutoModelForSeq2SeqLM
import torch


2024-01-27 20:03:20.258101: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-27 20:03:20.557713: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-27 20:03:20.557760: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-27 20:03:20.634124: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-27 20:03:21.955052: W tensorflow/stream_executor/pla

In [27]:
!pip install datasets


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [23]:
from datasets import load_dataset

In [29]:
# Load dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

ValueError: BuilderConfig '3.0.0' not found. Available: ['default']

In [ ]:
!pip install sentencepiece

In [ ]:
# Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [ ]:
# Preprocess function
def preprocess(example):
    inputs = tokenizer(example["article"], max_length=512, return_tensors="pt", padding=True, truncation=True)
    outputs = tokenizer(example["highlights"], max_length=150, return_tensors="pt", padding=True, truncation=True)
    return {
        "input_ids": inputs["input_ids"].squeeze().to(torch.long),
        "attention_mask": inputs["attention_mask"].squeeze().to(torch.long),
        "labels": outputs["input_ids"].squeeze().to(torch.long),
    }


In [ ]:
# Apply preprocessing to the dataset
train_data = dataset["train"].map(preprocess, batched=True)

Map: 100%|██████████| 287113/287113 [53:32<00:00, 89.37 examples/s]


In [ ]:
# Initialize T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-base")

model.safetensors: 100%|██████████| 892M/892M [00:04<00:00, 215MB/s]
generation_config.json: 100%|██████████| 147/147 [00:00<00:00, 11.8kB/s]


In [ ]:
# Training loop
num_epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

for epoch in range(num_epochs):
    model.train()
    for batch in train_data:
        # Filter out non-tensor values from the batch
        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}

        # Continue only if there are tensor values in the batch
        if any(isinstance(v, torch.Tensor) for v in batch.values()):
            optimizer.zero_grad()
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()


In [ ]:
# Save the trained model
model.save_pretrained("personalized_summarizer_model")

In [ ]:
tokenizer.save_pretrained("personalized_tokenizer")

('personalized_tokenizer/tokenizer_config.json',
 'personalized_tokenizer/special_tokens_map.json',
 'personalized_tokenizer/spiece.model',
 'personalized_tokenizer/added_tokens.json')

In [ ]:
trained_model = T5ForConditionalGeneration.from_pretrained("personalized_summarizer_model")
saved_tokenizer = T5Tokenizer.from_pretrained("personalized_tokenizer")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!pip install pipeline


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
pipe = pipeline('summarization', model=trained_model, tokenizer=saved_tokenizer)

In [8]:
input_text=dataset['train'][8]['article'][:1000]

In [ ]:
inputs = saved_tokenizer("summarize: " + input_text, return_tensors="tf", max_length=512, truncation=True)

2024-01-27 19:28:45.664392: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-01-27 19:28:45.664468: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-01-27 19:28:45.664499: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-c834d4ff-69a4-4f06-9794-5b50ea439d84): /proc/driver/nvidia/version does not exist
2024-01-27 19:28:45.671532: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
summery=pipe(input_text)

In [ ]:
print(summery)

[{'summary_text': 'mentally ill inmates are housed on the "forgotten floor" of a pretrial detention facility . most often, they face drug charges or charges of assaulting an officer . mentally ill people often won\'t do what they\'re told when police arrive on the scene .'}]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c834d4ff-69a4-4f06-9794-5b50ea439d84' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>